<a href="https://colab.research.google.com/github/ayobellobia/BIA-AI-Assignments/blob/main/Project%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2: Data Cleaning and Preparation

# Task 1: Advanced Data Cleaning

In [1]:
#Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
# Load dataset

prices = pd.read_csv('historical_stock_prices.csv')
stock = pd.read_csv('historical_stocks.csv')

print(f"'stock' DataFrame has {stock.shape[0]} rows and {stock.shape[1]} columns.")
print(f"'prices' DataFrame has {prices.shape[0]} rows and {prices.shape[1]} columns.")

'stock' DataFrame has 6460 rows and 5 columns.
'prices' DataFrame has 10597 rows and 8 columns.


In [3]:
# Clean whitespace
stock.columns = stock.columns.str.strip()
prices.columns = prices.columns.str.strip()

# Drop bad rows
stock = stock.dropna()
prices = prices.dropna()

# Convert safely
prices['date'] = pd.to_datetime(prices['date'], errors='coerce')

# Merge dataset on ticker
df_merged = prices.merge(stock, on='ticker', how='left')

# Align index with prices
df_merged.index = prices.index

print("Merged Dataset Preview:\n")
print(df_merged.head())

Merged Dataset Preview:

  ticker   open  close  adj_close    low   high   volume       date exchange  \
0    AHH  11.50  11.58   8.493155  11.25  11.68  4633900 2013-05-08     NYSE   
1    AHH  11.66  11.55   8.471151  11.50  11.66   275800 2013-05-09     NYSE   
2    AHH  11.55  11.60   8.507822  11.50  11.60   277100 2013-05-10     NYSE   
3    AHH  11.63  11.65   8.544494  11.55  11.65   147400 2013-05-13     NYSE   
4    AHH  11.60  11.53   8.456484  11.50  11.60   184100 2013-05-14     NYSE   

                              name   sector     industry  
0  ARMADA HOFFLER PROPERTIES, INC.  FINANCE  REAL ESTATE  
1  ARMADA HOFFLER PROPERTIES, INC.  FINANCE  REAL ESTATE  
2  ARMADA HOFFLER PROPERTIES, INC.  FINANCE  REAL ESTATE  
3  ARMADA HOFFLER PROPERTIES, INC.  FINANCE  REAL ESTATE  
4  ARMADA HOFFLER PROPERTIES, INC.  FINANCE  REAL ESTATE  


Advanced Data Cleaning

In [4]:
# Handle Missing Values
# Check missing values before cleaning
print("Missing Values Before Cleaning:\n")
print(df_merged.isnull().sum())

Missing Values Before Cleaning:

ticker          0
open            0
close           0
adj_close       0
low             0
high            0
volume          0
date            1
exchange     1458
name         1458
sector       1458
industry     1458
dtype: int64


In [5]:
#Handle missing values
# Forward fill numerical
# Identify numeric and categorical columns
num_cols = df_merged.select_dtypes(include=[np.number]).columns
cat_cols = df_merged.select_dtypes(include=["object"]).columns

# Interpolate numeric columns
df_merged[num_cols] = df_merged[num_cols].interpolate(method='linear')

# Fill categorical NaNs with "Unknown"
df_merged[cat_cols] = df_merged[cat_cols].fillna("Unknown")

# Check missing values after cleaning
print("Missing Values After Cleaning")
print(df_merged.isnull().sum())

Missing Values After Cleaning
ticker       0
open         0
close        0
adj_close    0
low          0
high         0
volume       0
date         1
exchange     0
name         0
sector       0
industry     0
dtype: int64


In [6]:
#Outliers detection and handlig

# Outlier detection and handling for 'close' and 'volume
# IQR-based outlier removal function
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    filtered_df = df[(df[column] >= lower) & (df[column] <= upper)]
    print(f"{column}: Removed {len(df) - len(filtered_df)} outliers")
    return filtered_df

# Before removal
print("Before outlier removal:", df_merged.shape)

# Sequentially remove outliers
merged = remove_outliers_iqr(df_merged, 'close')
merged = remove_outliers_iqr(merged, 'volume')

# After removal
print("After outlier removal:", merged.shape)

Before outlier removal: (10597, 12)
close: Removed 37 outliers
volume: Removed 1577 outliers
After outlier removal: (8983, 12)


In [7]:
#Error Identification: Replacing negative values and then interpolate
for col in ['open','high','low','close','volume']:
    neg_count = (merged[col] < 0).sum()
    print(f"{col}: Replacing {neg_count} negative values with NaN")
    merged.loc[merged[col] < 0, col] = np.nan
    merged[col] = merged[col].interpolate()

open: Replacing 0 negative values with NaN
high: Replacing 0 negative values with NaN
low: Replacing 0 negative values with NaN
close: Replacing 0 negative values with NaN
volume: Replacing 0 negative values with NaN


# Task 2: Data Transformation

In [8]:
# Feature Engineering
# Sorting based on ticker
merged = merged.sort_values(by=['ticker', 'date'])
print("DataFrame sorted by ticker and date.")
merged.head()

# Calculating 7-day and 30-day averages
merged['7_day_MA'] = merged.groupby('ticker')['close'].transform(lambda x: x.rolling(window=7).mean())
merged['30_day_MA'] = merged.groupby('ticker')['close'].transform(lambda x: x.rolling(window=30).mean())
print("Calculated 7-day and 30-day rolling averages.")
merged.tail()

# Daily Returns
merged['daily_return'] = merged['close'].pct_change()
# Future close price (for ML prediction)
merged['future_close_7'] = merged['close'].shift(-7)
merged.tail()

DataFrame sorted by ticker and date.
Calculated 7-day and 30-day rolling averages.


,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry,7_day_MA,30_day_MA,daily_return,future_close_7
10475,VIAV,4.835040,4.799488,4.799488,4.728384,4.906143,315700.0,1995-10-03,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.593796,3.923137,-0.007353,NaN
10497,VIAV,4.657281,4.728384,4.728384,4.657281,4.763937,494300.0,1995-10-04,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.629347,3.977650,-0.014815,NaN
10519,VIAV,4.692833,4.497298,4.497298,4.443970,4.692833,248200.0,1995-10-05,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.649663,4.025645,-0.048872,NaN
10541,VIAV,4.586177,4.550626,4.550626,4.479522,4.586177,143400.0,1995-10-06,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.675057,4.074232,0.011858,NaN
10563,VIAV,4.550626,4.106229,4.106229,4.052901,4.550626,679200.0,1995-10-09,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.667439,4.110376,-0.097656,NaN


In [9]:
# Calculate the 30-day rolling standard deviation of the 'close' price, grouped by 'ticker', to serve as a volatility measure
merged['30_day_STD'] = merged.groupby('ticker')['close'].transform(lambda x: x.rolling(window=30).std())
print("Calculated 30-day rolling standard deviation.")
merged.tail()

Calculated 30-day rolling standard deviation.


,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry,7_day_MA,30_day_MA,daily_return,future_close_7,30_day_STD
10475,VIAV,4.835040,4.799488,4.799488,4.728384,4.906143,315700.0,1995-10-03,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.593796,3.923137,-0.007353,NaN,0.612922
10497,VIAV,4.657281,4.728384,4.728384,4.657281,4.763937,494300.0,1995-10-04,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.629347,3.977650,-0.014815,NaN,0.609258
10519,VIAV,4.692833,4.497298,4.497298,4.443970,4.692833,248200.0,1995-10-05,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.649663,4.025645,-0.048872,NaN,0.590699
10541,VIAV,4.586177,4.550626,4.550626,4.479522,4.586177,143400.0,1995-10-06,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.675057,4.074232,0.011858,NaN,0.570958
10563,VIAV,4.550626,4.106229,4.106229,4.052901,4.550626,679200.0,1995-10-09,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.667439,4.110376,-0.097656,NaN,0.535248


In [10]:
# Feature Engineering
# Sorting based on ticker
merged = merged.sort_values(by=['ticker', 'date'])
print("DataFrame sorted by ticker and date.")
merged.head()

# Calculating 7-day and 30-day averages
merged['7_day_MA'] = merged.groupby('ticker')['close'].transform(lambda x: x.rolling(window=7).mean())
merged['30_day_MA'] = merged.groupby('ticker')['close'].transform(lambda x: x.rolling(window=30).mean())
print("Calculated 7-day and 30-day rolling averages.")
merged.tail()

# Daily Returns
merged['daily_return'] = merged.groupby('ticker')['close'].pct_change()
# Future close price (for ML prediction)
merged['future_close_7'] = merged.groupby('ticker')['close'].shift(-7)
merged.tail()

DataFrame sorted by ticker and date.
Calculated 7-day and 30-day rolling averages.


,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry,7_day_MA,30_day_MA,daily_return,future_close_7,30_day_STD
10475,VIAV,4.835040,4.799488,4.799488,4.728384,4.906143,315700.0,1995-10-03,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.593796,3.923137,-0.007353,NaN,0.612922
10497,VIAV,4.657281,4.728384,4.728384,4.657281,4.763937,494300.0,1995-10-04,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.629347,3.977650,-0.014815,NaN,0.609258
10519,VIAV,4.692833,4.497298,4.497298,4.443970,4.692833,248200.0,1995-10-05,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.649663,4.025645,-0.048872,NaN,0.590699
10541,VIAV,4.586177,4.550626,4.550626,4.479522,4.586177,143400.0,1995-10-06,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.675057,4.074232,0.011858,NaN,0.570958
10563,VIAV,4.550626,4.106229,4.106229,4.052901,4.550626,679200.0,1995-10-09,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.667439,4.110376,-0.097656,NaN,0.535248


In [11]:
# Calculate the 14-day Relative Strength Index (RSI)
def calculate_rsi(data, window=14):
    delta = data.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.ewm(com=window - 1, adjust=False).mean()
    avg_loss = loss.ewm(com=window - 1, adjust=False).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

merged['RSI'] = merged.groupby('ticker')['close'].transform(calculate_rsi)
print("Calculated 14-day Relative Strength Index (RSI).")
merged.tail()

Calculated 14-day Relative Strength Index (RSI).


,ticker,open,close,adj_close,low,high,volume,date,exchange,name,sector,industry,7_day_MA,30_day_MA,daily_return,future_close_7,30_day_STD,RSI
10475,VIAV,4.835040,4.799488,4.799488,4.728384,4.906143,315700.0,1995-10-03,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.593796,3.923137,-0.007353,NaN,0.612922,63.914688
10497,VIAV,4.657281,4.728384,4.728384,4.657281,4.763937,494300.0,1995-10-04,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.629347,3.977650,-0.014815,NaN,0.609258,62.099239
10519,VIAV,4.692833,4.497298,4.497298,4.443970,4.692833,248200.0,1995-10-05,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.649663,4.025645,-0.048872,NaN,0.590699,56.483908
10541,VIAV,4.586177,4.550626,4.550626,4.479522,4.586177,143400.0,1995-10-06,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.675057,4.074232,0.011858,NaN,0.570958,57.440330
10563,VIAV,4.550626,4.106229,4.106229,4.052901,4.550626,679200.0,1995-10-09,NASDAQ,VIAVI SOLUTIONS INC.,TECHNOLOGY,SEMICONDUCTORS,4.667439,4.110376,-0.097656,NaN,0.535248,47.977128


In [12]:
# Summary of New Engineered Features
print("Summary of newly engineered features:")
print(merged[['7_day_MA', '30_day_MA', '30_day_STD', 'RSI']].describe())

Summary of newly engineered features:
          7_day_MA    30_day_MA   30_day_STD          RSI
count  8916.000000  8663.000000  8663.000000  8968.000000
mean     11.367743    11.359308     0.535557    49.873871
std       6.730657     6.687008     0.506088    13.042604
min       0.127727     0.133620     0.001542     0.000000
25%       6.064286     6.186000     0.192121    41.059832
50%      11.507857    11.577083     0.377281    49.731450
75%      15.232143    15.054333     0.717735    58.346123
max      32.014286    30.813667     3.472112   100.000000


In [13]:
#Data Normalization/Standardization
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_cols = ['open', 'high', 'close', 'volume','7_day_MA', '30_day_MA', '30_day_STD', 'RSI']
merged[scaled_cols] = scaler.fit_transform(merged[scaled_cols])
print(merged.head())

     ticker      open     close  adj_close        low      high    volume  \
6850   AAPL -1.609369 -1.608434   0.023489   0.513393 -1.613487  1.477641   
1       AHH  0.041350  0.026609   8.471151  11.500000  0.017110  0.976520   
2       AHH  0.025060  0.034021   8.507822  11.500000  0.008322  0.985644   
3       AHH  0.036907  0.041432   8.544494  11.550000  0.015645  0.075344   
4       AHH  0.032464  0.023644   8.456484  11.500000  0.008322  0.332923   

           date exchange                             name      sector  \
6850 1983-01-11   NASDAQ                       APPLE INC.  TECHNOLOGY   
1    2013-05-09     NYSE  ARMADA HOFFLER PROPERTIES, INC.     FINANCE   
2    2013-05-10     NYSE  ARMADA HOFFLER PROPERTIES, INC.     FINANCE   
3    2013-05-13     NYSE  ARMADA HOFFLER PROPERTIES, INC.     FINANCE   
4    2013-05-14     NYSE  ARMADA HOFFLER PROPERTIES, INC.     FINANCE   

                    industry  7_day_MA  30_day_MA  daily_return  \
6850  COMPUTER MANUFACTURING   

In [14]:
# Categororical data encoding 0/1
cat_features = ['sector', 'industry', 'exchange']
merged_encoded = pd.get_dummies(
    merged,
    columns=cat_features,
    drop_first=True,
    dtype=int
)

print(merged_encoded.head())

     ticker      open     close  adj_close        low      high    volume  \
6850   AAPL -1.609369 -1.608434   0.023489   0.513393 -1.613487  1.477641   
1       AHH  0.041350  0.026609   8.471151  11.500000  0.017110  0.976520   
2       AHH  0.025060  0.034021   8.507822  11.500000  0.008322  0.985644   
3       AHH  0.036907  0.041432   8.544494  11.550000  0.015645  0.075344   
4       AHH  0.032464  0.023644   8.456484  11.500000  0.008322  0.332923   

           date                             name  7_day_MA  ...  \
6850 1983-01-11                       APPLE INC.       NaN  ...   
1    2013-05-09  ARMADA HOFFLER PROPERTIES, INC.       NaN  ...   
2    2013-05-10  ARMADA HOFFLER PROPERTIES, INC.       NaN  ...   
3    2013-05-13  ARMADA HOFFLER PROPERTIES, INC.       NaN  ...   
4    2013-05-14  ARMADA HOFFLER PROPERTIES, INC.       NaN  ...   

      industry_INVESTMENT MANAGERS  industry_MEDICAL SPECIALITIES  \
6850                             0                              0

# Task 4: Integration and Formatting for Modeling

In [15]:
# Consolidate Data: Ensure all data transformations and cleanings are integrated into a single, coherent dataset ready for analysis
clean_data = merged_encoded.copy()
clean_data.dropna(inplace=True)

print(f"Consolidated Data: {clean_data.shape[0]} rows and {clean_data.shape[1]} columns.")
print("Consolidated Data Preview:")
print(clean_data.head())


Consolidated Data: 8586 rows and 32 columns.
Consolidated Data Preview:
   ticker      open     close  adj_close    low      high    volume  \
30    AHH -0.019368 -0.035651   8.163107  11.13 -0.035617  0.466274   
31    AHH -0.031215 -0.014898   8.265792  11.12 -0.040010  1.061444   
32    AHH -0.026772 -0.022309   8.229120  10.70 -0.037081  0.104120   
33    AHH -0.014925  0.001408   8.346467  10.98 -0.020970 -0.288215   
34    AHH  0.008769  0.001408   8.346467  10.90 -0.012183  0.261334   

         date                             name  7_day_MA  ...  \
30 2013-06-20  ARMADA HOFFLER PROPERTIES, INC.  0.007340  ...   
31 2013-06-21  ARMADA HOFFLER PROPERTIES, INC.  0.002670  ...   
32 2013-06-24  ARMADA HOFFLER PROPERTIES, INC. -0.003061  ...   
33 2013-06-25  ARMADA HOFFLER PROPERTIES, INC. -0.004547  ...   
34 2013-06-26  ARMADA HOFFLER PROPERTIES, INC. -0.008155  ...   

    industry_INVESTMENT MANAGERS  industry_MEDICAL SPECIALITIES  \
30                             0           

Data Splitting

In [16]:
# Data Splitting: Prepare the data for predictive modeling by splitting it into training, validation, and test sets.
from sklearn.model_selection import train_test_split

y = clean_data['future_close_7']
X = clean_data.drop(columns=['future_close_7', 'ticker', 'date', 'adj_close', 'name'])

print(f"Features (X) shape: {X.shape}")
print(f"Target (y) shape: {y.shape}")

Features (X) shape: (8586, 27)
Target (y) shape: (8586,)


In [17]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (6010, 27)
y_train shape: (6010,)
X_val shape: (1288, 27)
y_val shape: (1288,)
X_test shape: (1288, 27)
y_test shape: (1288,)


In [18]:
#Save clean data: Save cleaned data and the splits for future us
clean_data.to_csv("clean_stock_data.csv", index=True)
X_train.to_csv("train_X.csv")
y_train.to_csv("train_y.csv")
X_val.to_csv("val_X.csv")
y_val.to_csv("val_y.csv")
X_test.to_csv("test_X.csv")
y_test.to_csv("test_y.csv")

## Summarry of Data Loading and Initial State

The `historical_stock_prices.csv` dataset was loaded into a DataFrame named `prices` and the `historical_stocks.csv` dataset was loaded into a DataFrame named `stock`.

**Initial State:**

*   **`prices` DataFrame:**
    *   **Rows:** 10597
    *   **Columns:** 8
    *   **Column Names:** `ticker`, `open`, `close`, `adj_close`, `low`, `high`, `volume`, `date`

*   **`stock` DataFrame:**
    *   **Rows:** 6460
    *   **Columns:** 5
    *   **Column Names:** `ticker`, `exchange`, `name`, `sector`, `industry`

## Summarize Initial Data Cleaning and Merging

## Initial Data Cleaning and Merging Summary

1.  **Whitespace Removal from Column Names**: Both the `stock` and `prices` DataFrames had their column names cleaned by removing leading/trailing whitespace using `df.columns.str.strip()`. This ensures consistency and prevents potential issues during data manipulation.

2.  **Handling Missing Rows**: Rows containing any missing values (NaN) were dropped from both the `stock` and `prices` DataFrames using the `dropna()` method. This step ensures that subsequent operations are performed on complete records.

3.  **Safe Date Column Conversion**: The 'date' column in the `prices` DataFrame was safely converted to datetime objects using `pd.to_datetime()`. The `errors='coerce'` argument was used to convert any unparseable dates into `NaT` (Not a Time), preventing errors and allowing for easier identification of invalid date entries.

4.  **Merging DataFrames**: The `prices` and `stock` DataFrames were merged into a new DataFrame called `df_merged`. A `left` merge was performed on the 'ticker' column, meaning all rows from the `prices` DataFrame were kept, and matching information from the `stock` DataFrame was added based on the 'ticker'.

5.  **Index Alignment**: After the merge, the index of the `df_merged` DataFrame was reset to align with the original index of the `prices` DataFrame. This ensures that the merged DataFrame maintains the original order and indexing from the `prices` data, which is often crucial for time-series analysis or when relating back to the original granular data.

## Summary of Missing Value Handling

### Missing Value Handling Approach

**1. Initial Missing Values (from cell `B8l3728vfDcO`):**

Before cleaning, the `df_merged` DataFrame had the following missing values:

- `date`: 1 missing value
- `exchange`: 1458 missing values
- `name`: 1458 missing values
- `sector`: 1458 missing values
- `industry`: 1458 missing values

**2. Handling Strategy (from cell `0AnKXzYhhDVw`):**

The approach taken was to differentiate between numerical and categorical columns for handling missing values:

- **Numerical Columns:** For columns identified as numerical (e.g., 'open', 'close', 'adj_close', 'low', 'high', 'volume'), missing values were filled using **linear interpolation**. This method estimates missing values based on the values before and after them in the sequence.

- **Categorical Columns:** For columns identified as categorical (e.g., 'exchange', 'name', 'sector', 'industry'), missing values were filled with the string **"Unknown"**.

**3. Missing Values After Cleaning (from cell `0AnKXzYhhDVw`):**

After applying these cleaning steps, the missing values were reduced significantly:

- `date`: 1 missing value (remained after cleaning)
- All other columns (`ticker`, `open`, `close`, `adj_close`, `low`, `high`, `volume`, `exchange`, `name`, `sector`, `industry`) had **0 missing values**.

This indicates that the date column's missing value was not addressed by the interpolation or fillna("Unknown") methods, likely due to its non-numeric, non-standard categorical nature in the initial classification, or being the first/last record for its group during interpolation.


## Summary of Outlier Detection and Correction

### Explanation of IQR-based Outlier Detection and Correction

The Interquartile Range (IQR) method was used to identify and remove outliers from the 'close' and 'volume' columns of the dataset. This method is robust to skewed data and is defined as the range between the first quartile (Q1) and the third quartile (Q3).

1.  **IQR-based Outlier Removal Function (`remove_outliers_iqr`)**: This function calculates the first quartile (Q1, 25th percentile) and the third quartile (Q3, 75th percentile) for a given column. The IQR is then computed as `Q3 - Q1`. The lower bound for outlier detection is defined as `Q1 - 1.5 * IQR`, and the upper bound is `Q3 + 1.5 * IQR`. Any data point falling outside these bounds is considered an outlier and is removed.

2.  **Columns Applied**: This method was applied sequentially to the 'close' and 'volume' columns.

3.  **Initial Dataset Size**: Before outlier removal, the `df_merged` DataFrame had `(10597, 12)` rows and columns.

4.  **Outliers Removed**:
    *   For the 'close' column: `37` outliers were removed.
    *   For the 'volume' column: `1577` outliers were removed.

5.  **Final Dataset Size**: After removing outliers from both columns, the `merged` DataFrame had `(8983, 12)` rows and columns, indicating a reduction in dataset size from 10597 rows to 8983 rows.

## Summary of Error Correction for Negative Values

### Instructions
1. Explain that the financial columns 'open', 'high', 'low', 'close', and 'volume' were checked for negative values.
2. Describe the method used to correct these values: replacing any negative values with NaN, and then using interpolation to fill these NaN values.
3. State the outcome of this correction step, noting that no negative values were found in these columns based on the code's output.

To ensure data integrity and prevent issues with financial calculations and modeling, the following financial columns were explicitly checked for negative values: `open`, `high`, `low`, `close`, and `volume`.

The correction method involved a two-step process:
1.  **Identification and Replacement**: For each identified column, any value less than 0 was replaced with `np.nan` (Not a Number).
2.  **Interpolation**: The newly introduced `NaN` values (or any pre-existing `NaN` values) in these columns were then filled using linear interpolation. This method estimates missing values based on surrounding valid data points, maintaining the time-series integrity of the stock data.

Upon executing the correction code, it was observed that **no negative values were found** in the `open`, `high`, `low`, `close`, or `volume` columns. The output from the code (`Replacing 0 negative values with NaN`) confirms that all values in these critical financial metrics were already non-negative before the correction process was applied. This indicates a relatively clean dataset concerning this specific type of error.

## Summarry of Feature Engineering Techniques

#### Engineered Features Overview:

1.  **7-day Moving Average ('7_day_MA') and 30-day Moving Average ('30_day_MA'):**
    *   **Creation:** These features are calculated by taking the rolling mean of the `'close'` price over a 7-day and 30-day window, respectively. The `groupby('ticker')` and `transform()` methods ensure these calculations are performed independently for each stock, preventing data leakage across different tickers.
    *   **Purpose:** Moving averages are widely used technical indicators to smooth out price data and identify trends. The 7-day MA represents short-term trends, while the 30-day MA indicates longer-term trends. Comparing these can help identify potential trend changes.

2.  **Daily Returns ('daily_return'):**
    *   **Creation:** Calculated as the percentage change of the `'close'` price from one day to the next, specifically for each ticker using `groupby('ticker')['close'].pct_change()`.
    *   **Purpose:** Daily returns quantify the daily price fluctuation of a stock. It's a fundamental metric for assessing volatility and performance over short periods.

3.  **Future Close Price ('future_close_7'):**
    *   **Creation:** This feature captures the `'close'` price 7 days into the future, relative to the current row, grouped by `'ticker'` using `groupby('ticker')['close'].shift(-7)`.
    *   **Purpose:** This serves as the target variable for predictive modeling. The goal is to predict the stock's closing price 7 days ahead, which is crucial for forecasting applications.

4.  **30-day Rolling Standard Deviation ('30_day_STD'):**
    *   **Creation:** Computed as the rolling standard deviation of the `'close'` price over a 30-day window, again grouped by `'ticker'` to ensure ticker-specific volatility.
    *   **Purpose:** The 30-day rolling standard deviation is a key measure of price volatility. A higher standard deviation indicates greater price dispersion and, consequently, higher risk or potential reward over the 30-day period.

5.  **14-day Relative Strength Index (RSI):**
    *   **Creation:** The RSI is a momentum oscillator that measures the speed and change of price movements. It's calculated over a 14-day period. The calculation involves first determining daily price changes, then separating gains and losses, and finally calculating exponential moving averages of these gains and losses to derive the RSI value. This is applied independently to each ticker using `groupby('ticker')['close'].transform(calculate_rsi)`.
    *   **Purpose:** RSI is used to identify overbought or oversold conditions in the market. Typically, an RSI above 70 suggests the asset is overbought (potentially due for a price correction), while an RSI below 30 suggests it's oversold (potentially due for a rebound).

These features are specifically designed to capture different aspects of stock price behavior—trends, volatility, momentum, and future outcomes—all crucial for building robust predictive models in finance. The use of `groupby('ticker')` with `transform()` ensures that these financial metrics are calculated accurately for each individual stock, maintaining the integrity of the time-series data.

## Summary of Data Scaling and Encoding Technique

### Numerical Feature Scaling
Numerical features in the dataset were scaled using `StandardScaler` from the `sklearn.preprocessing` library. This method transforms the data such that its mean is 0 and its standard deviation is 1, which helps in preparing numerical features for machine learning models by bringing them to a similar scale. The specific columns that underwent this scaling were:

*   `open`
*   `high`
*   `close`
*   `volume`
*   `7_day_MA` (7-day Moving Average)
*   `30_day_MA` (30-day Moving Average)
*   `30_day_STD` (30-day Standard Deviation)
*   `RSI` (Relative Strength Index)

### Categorical Feature Encoding
Categorical features were encoded using one-hot encoding with the `pd.get_dummies()` function from the pandas library. This technique converts categorical variables into a format that can be provided to ML algorithms to do a better job in prediction. Each category value is converted into a new column, and a 1 or 0 value is assigned to the column. The columns that were encoded are:

*   `sector`
*   `industry`
*   `exchange`

To avoid multicollinearity, the `drop_first=True` parameter was utilized during one-hot encoding. This drops the first category for each feature, reducing redundancy and preventing issues in downstream modeling.

## Summary of Final Data Preparation

The final stage of data preparation involved consolidating all the cleaned and transformed features into a single dataset, handling any remaining missing values, and then splitting this data into distinct sets for model training, validation, and testing.

### Data Consolidation

1.  **`clean_data = merged_encoded.copy()`**: A new DataFrame named `clean_data` was created as a direct copy of `merged_encoded`. This step ensured that all previously applied data cleaning, feature engineering (like rolling averages, RSI, daily returns), scaling, and one-hot encoding for categorical features were integrated into one comprehensive dataset. This consolidated DataFrame serves as the final input for model development.

### Handling Remaining Missing Values

2.  **`clean_data.dropna(inplace=True)`**: After the feature engineering steps, some new columns (e.g., `7_day_MA`, `30_day_MA`, `30_day_STD`, `RSI`, `daily_return`, `future_close_7`) contained `NaN` values, primarily due to the rolling window calculations at the beginning of each ticker's time series or the `shift` operation for `future_close_7` at the end. The `dropna()` method was applied to `clean_data` to remove all rows that still contained any missing values. This step was crucial to ensure that the dataset used for modeling was complete and free of `NaN`s, which most machine learning algorithms cannot handle directly. The output `Consolidated Data: 8586 rows and 32 columns.` indicates that `8983 - 8586 = 397` rows were dropped during this process.

### Data Splitting for Predictive Modeling

3.  **Defining Target and Features**: The data was prepared for predictive modeling by defining the target variable (`y`) and the feature set (`X`).
    *   `y = clean_data['future_close_7']`: The target variable `y` was identified as `'future_close_7'`, which represents the close price 7 days into the future, intended for forecasting.
    *   `X = clean_data.drop(columns=['future_close_7', 'ticker', 'date', 'adj_close', 'name'])`: The feature set `X` was created by dropping the target variable itself (`'future_close_7'`) and other columns not suitable as features: `'ticker'` (identifier), `'date'` (datetime object, though date information could be extracted for features), `'adj_close'` (highly correlated with `'close'`), and `'name'` (textual identifier).

4.  **Splitting Ratios**: The dataset was then split into three distinct sets: training, validation, and test, to facilitate robust model evaluation and hyperparameter tuning.
    *   Initially, `X` and `y` were split into `X_train`, `y_train` (70% of the data) and `X_temp`, `y_temp` (30% of the data) using `train_test_split` with `test_size=0.3`.
    *   Subsequently, `X_temp` and `y_temp` were further split into `X_val`, `y_val` (15% of the original data) and `X_test`, `y_test` (15% of the original data) using `test_size=0.5`.
    *   This results in a 70% training set, 15% validation set, and 15% test set, ensuring a proper separation of data for unbiased model evaluation.

5.  **Final Shapes**: The shapes of the resulting datasets after splitting are:
    *   `X_train` shape: (6010, 27)
    *   `y_train` shape: (6010,)
    *   `X_val` shape: (1288, 27)
    *   `y_val` shape: (1288,)
    *   `X_test` shape: (1288, 27)
    *   `y_test` shape: (1288,)

These steps ensure that the data is well-prepared, consistent, and structured appropriately for subsequent machine learning model training and evaluation.


## Summary Report:

### Data Analysis Key Findings

*   **Data Loading and Initial State**:
    *   `historical_stock_prices.csv` was loaded into a `prices` DataFrame with 10,597 rows and 8 columns (`ticker`, `open`, `close`, `adj_close`, `low`, `high`, `volume`, `date`).
    *   `historical_stocks.csv` was loaded into a `stock` DataFrame with 6,460 rows and 5 columns (`ticker`, `exchange`, `name`, `sector`, `industry`).
*   **Initial Data Cleaning and Merging**:
    *   Whitespace was removed from column names in both DataFrames.
    *   Rows with any missing values were dropped from `stock` and `prices`.
    *   The `date` column in `prices` was safely converted to datetime objects, coercing errors.
    *   `prices` and `stock` DataFrames were merged into `df_merged` using a left merge on the `ticker` column.
    *   The index of `df_merged` was reset to align with the original `prices` DataFrame.
*   **Missing Value Handling**:
    *   Numerical features had missing values filled using linear interpolation.
    *   Categorical features (`exchange`, `name`, `sector`, `industry`) had missing values filled with "Unknown".
    *   One missing value in the `date` column persisted after these steps.
    *   Initially, 1458 missing values were present across `exchange`, `name`, `sector`, `industry` in `df_merged`. After handling, these columns had 0 missing values.
*   **Outlier Detection and Correction**:
    *   The Interquartile Range (IQR) method was applied to the 'close' and 'volume' columns to remove outliers.
    *   `37` outliers were removed from the 'close' column.
    *   `1577` outliers were removed from the 'volume' column.
    *   The dataset size was reduced from 10,597 rows to 8,983 rows after outlier removal.
*   **Error Correction for Negative Values**:
    *   A process was defined to replace negative values in 'open', 'high', 'low', 'close', and 'volume' with `NaN` and then interpolate them.
    *   However, during execution, **no negative values were found** in these columns, indicating a clean dataset in this regard.
*   **Feature Engineering Techniques**:
    *   **7-day and 30-day Moving Averages (`7_day_MA`, `30_day_MA`)**: Rolling means of the 'close' price to identify short-term and long-term trends.
    *   **Daily Returns (`daily_return`)**: Percentage change of 'close' price for daily price fluctuation and volatility.
    *   **Future Close Price (`future_close_7`)**: The 'close' price 7 days into the future, serving as the target variable for predictive modeling.
    *   **30-day Rolling Standard Deviation (`30_day_STD`)**: Rolling standard deviation of 'close' price to measure price volatility over 30 days.
    *   **14-day Relative Strength Index (RSI)**: A momentum oscillator to identify overbought or oversold conditions.
    *   All these features were calculated independently for each stock using `groupby('ticker')` and `transform()`.
*   **Data Scaling and Encoding**:
    *   Numerical features (`open`, `high`, `close`, `volume`, `7_day_MA`, `30_day_MA`, `30_day_STD`, `RSI`) were scaled using `StandardScaler` to have a mean of 0 and a standard deviation of 1.
    *   Categorical features (`sector`, `industry`, `exchange`) were encoded using one-hot encoding with `pd.get_dummies()`, utilizing `drop_first=True` to prevent multicollinearity.
*   **Final Data Preparation**:
    *   All cleaned and transformed data were consolidated into a `clean_data` DataFrame.
    *   Any remaining missing values (primarily from rolling window calculations and `future_close_7`) were removed using `dropna()`, resulting in a final dataset of 8,586 rows from the previous 8,983 rows (397 rows dropped).
    *   The target variable `y` was set to `future_close_7`.
    *   The feature set `X` included all columns except `future_close_7`, `ticker`, `date`, `adj_close`, and `name`.
    *   The data was split into training (70%), validation (15%), and test (15%) sets using `train_test_split`. The final sizes were `X_train` (6010, 27), `X_val` (1288, 27), `X_test` (1288, 27).

